In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tmdbsimple as tmdb
import os
import re
import sys
import pickle
import time
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../src'))

In [2]:
with open('../Data/api_key.pkl', 'rb') as hnd:
    tmdb.API_KEY = pickle.load(hnd)['api_key']

In [3]:
data = pd.read_csv('../Data/NetflixViewingHistory.csv')
import gather_data_deprecated as eda1
netflix_df = eda1.parse_netflix(data)

Total number of TV Show + Movies:  1405
TV Show vs Movie
Dataframe shape:  (1405, 6)


In [4]:
shows = netflix_df[netflix_df['TV_Show_flag'] == 'TV_Show']
movies = netflix_df[netflix_df['TV_Show_flag'] == 'Movie']

In [5]:
search= tmdb.Search()

## Begin looking into TV Shows 

The plan is to find the name and season of the show and find the match in tmdb. We can get the names of all the episodes in the season and find the TMDB id number from there. After we have the id number, we query using the tmdb tv search and get the run time and genre. There's plenty of info on the way  

Steps: 
1. Find TV Show name and season 
2. Find episode match in Season API call 
3. Get the ID from the season API 
4. Query using TV to get the run time + other info 

In [6]:
shows.shape

(1357, 6)

In [7]:
shows.head()

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag
0,Dating Around: Season 1: Gurki,2019-03-03,Dating Around,Season 1,Gurki,TV_Show
1,Dating Around: Season 1: Luke,2019-03-03,Dating Around,Season 1,Luke,TV_Show
2,The Umbrella Academy: Season 1: We Only See Ea...,2019-02-19,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV_Show
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show


In [8]:
temp_season_numb = shows['Season'].str.split('Season ', n=2, expand=True)[1]

In [9]:
# Fix this regex 
shows["Season #2"] = temp_season_numb.str.extract('^(\d{0,2}$)?')

/home/owner/Documents/Github_projects/Netflix_TMDB/venv1/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
shows["Season #2"].value_counts()

1     318
3     189
2     186
4     171
5     146
6     108
7      78
8      40
9      24
10     17
Name: Season #2, dtype: int64

In [11]:
count_per_tv_title = shows.groupby('Show Name').size().sort_values(ascending=False)
count_per_tv_title.head()

Show Name
Friends                 171
30 Rock                 134
The Office (U.S.)       115
Parks and Recreation     90
White Collar             81
dtype: int64

In [12]:
dups = shows.duplicated()
dups.sum()

0

In [13]:
unique_shows  = count_per_tv_title.index
show_name1 = unique_shows[0]
print(show_name1)

Friends


In [14]:
res1 = search.tv(query=show_name1)['results'][0]

In [15]:
res1_id = res1['id']

In [16]:
tv_show_ids = {}
for show_name in unique_shows:
    search_result = search.tv(query=show_name)
    if len(search_result['results']) >= 1:
        temp_id = search_result['results'][0]['id']
        tv_show_ids[show_name]=temp_id
        time.sleep(0.3)

In [17]:
len(unique_shows)

74

In [18]:
tv_show_ids_df = pd.Series(tv_show_ids, name='TMDB_id')

In [19]:
shows2 = pd.merge(shows, tv_show_ids_df, how='left', left_on='Show Name', right_index=True)

In [20]:
shows2.head()

,Title,Date,Show Name,Season,Episode Name,TV_Show_flag,Season #2,TMDB_id
0,Dating Around: Season 1: Gurki,2019-03-03,Dating Around,Season 1,Gurki,TV_Show,1,86807
1,Dating Around: Season 1: Luke,2019-03-03,Dating Around,Season 1,Luke,TV_Show,1,86807
2,The Umbrella Academy: Season 1: We Only See Ea...,2019-02-19,The Umbrella Academy,Season 1,We Only See Each Other at Weddings and Funerals,TV_Show,1,75006
4,American Vandal: Season 1: Nailed,2018-11-19,American Vandal,Season 1,Nailed,TV_Show,1,73126
5,American Vandal: Season 1: A Limp Alibi,2018-11-19,American Vandal,Season 1,A Limp Alibi,TV_Show,1,73126


In [21]:
tv_res1 = tmdb.TV_Seasons(res1_id, season_number=1).info()
tv_res1

{'_id': '52570fcd19c295731c020203',
 'air_date': '1994-09-22',
 'episodes': [{'air_date': '1994-09-22',
   'episode_number': 1,
   'id': 85987,
   'name': 'The Pilot',
   'overview': 'This is the one where it all began ... Rachel leaves Barry at the altar, meets the gang, and moves in with Monica. Monica, meanwhile, sleeps with Paul the Wine Guy, and Ross is reeling from his divorce from Carol, who has become a lesbian.',
   'production_code': '456650',
   'season_number': 1,
   'show_id': 1668,
   'still_path': '/fbtaoynlPpENx3Ss2laC7wgqLIP.jpg',
   'vote_average': 7.032,
   'vote_count': 31,
   'crew': [{'id': 157437,
     'credit_id': '52570fd019c295731c020600',
     'name': 'James Burrows',
     'department': 'Directing',
     'job': 'Director',
     'gender': 2,
     'profile_path': '/xZn77KyPoq3gebCNTM6X6rrDn1L.jpg'},
    {'id': 163461,
     'credit_id': '52570fd219c295731c0209fb',
     'name': 'Marta Kauffman',
     'department': 'Writing',
     'job': 'Writer',
     'gender': 1

## Really Just have to Parse TV response

In [22]:
# tv_query = tmdb.TV(id=int(bigger_temp_df.iloc[1]['id']))
tv_query = tmdb.TV(2004)

In [38]:
tv_query.info()

'Malcolm in the Middle'

In [24]:
shows2.columns

Index(['Title', 'Date', 'Show Name', 'Season', 'Episode Name', 'TV_Show_flag',
       'Season #2', 'TMDB_id'],
      dtype='object')

In [25]:
normal_movie_fields = ['homepage', 'imdb_id', 'overview', 'popularity' \
            , 'release_date', 'runtime', 'vote_average', 'vote_count']

In [26]:
set(normal_movie_fields).difference(set(tv_query.info().keys()))

{'imdb_id', 'release_date', 'runtime'}

In [27]:
normal_tv_fields = ['homepage', 'id', 'overview', 'popularity' \
                   , 'revenue', 'episode_run_time', 'vote_average', 'vote_count'
                   , 'in_production', 'type', 'status', 'number_of_seasons', 'number_of_episodes', 'networks']

In [28]:
set(normal_tv_fields).difference(set(tv_query.info().keys()))

{'revenue'}

In [29]:
smaller_parse = {k:v for k,v in tv_query.info().items() if k in normal_tv_fields}

In [30]:
shows2.columns

Index(['Title', 'Date', 'Show Name', 'Season', 'Episode Name', 'TV_Show_flag',
       'Season #2', 'TMDB_id'],
      dtype='object')

In [41]:
sm_unique_shows = pd.Series(shows2['TMDB_id'].unique()[:20], name='Show Name')

In [42]:
sm_unique_shows

0     86807
1     75006
2     73126
3      2316
4      8592
5     42009
6     68421
7     46952
8     66488
9     63351
10    67178
11    66732
12    66676
13     4608
14    79058
15     1668
16    71578
17    71308
18     1434
19    71683
Name: Show Name, dtype: int64

In [43]:
def apply_tv_parse_fx(tmdb_id):
    raw_tv_response = tmdb.TV(tmdb_id).info()
    
    shorter_response = {}
    
    # Responses that need to be parsed 
    shorter_response['n_production_companies'] = len(raw_tv_response['production_companies'])
    shorter_response['primary_production_co'] = '' if len(raw_tv_response['production_companies']) == 0 else raw_tv_response['production_companies'][0]['name']
    shorter_response['runtime'] = 0 if len(raw_tv_response) == 0 else raw_tv_response['episode_run_time'][0] 
    shorter_response['release_date'] = raw_tv_response['first_air_date']
    shorter_response['n_network'] = len(raw_tv_response['networks'])
    shorter_response['primary_network'] = '' if len(raw_tv_response['networks']) == 0 else raw_tv_response['networks'][0]['name']
    
    # Fill dictionary with other items directly 
    normal_tv_fields = ['homepage', 'overview', 'popularity' \
                        , 'vote_average', 'vote_count', 'name'
                        , 'in_production', 'type', 'status', 'number_of_seasons', 'number_of_episodes']
    for field in normal_tv_fields:
        shorter_response[field] = raw_tv_response[field]
    
    return(shorter_response)
        

In [44]:
apply_tv_parse_fx(2316).keys()

dict_keys(['n_production_companies', 'primary_production_co', 'runtime', 'imdb_id', 'release_date', 'n_network', 'primary_network', 'homepage', 'overview', 'popularity', 'vote_average', 'vote_count', 'name', 'in_production', 'type', 'status', 'number_of_seasons', 'number_of_episodes'])

In [45]:
demo1 = sm_unique_shows.apply(apply_tv_parse_fx)

In [46]:
df1 = pd.DataFrame(demo1.to_list())

In [48]:
df1.head()

,homepage,imdb_id,in_production,n_network,n_production_companies,name,number_of_episodes,number_of_seasons,overview,popularity,primary_network,primary_production_co,release_date,runtime,status,type,vote_average,vote_count
0,https://www.netflix.com/title/80204889,86807,True,1,0,Dating Around,6,1,"In each episode of flirtations and fails, one ...",0.915,Netflix,,2019-02-14,30,Returning Series,Reality,6.0,1
1,https://www.netflix.com/title/80186863,75006,True,1,2,The Umbrella Academy,10,1,A dysfunctional family of superheroes comes to...,64.690,Netflix,Dark Horse Entertainment,2019-02-15,60,Returning Series,Scripted,7.7,111
2,https://www.netflix.com/title/80117545,73126,False,1,2,American Vandal,16,2,A true-crime satire that explores the aftermat...,6.797,Netflix,3 Arts Entertainment,2017-09-15,35,Ended,Miniseries,7.9,73
3,http://www.nbc.com/The_Office/,2316,False,1,0,The Office,195,9,The everyday lives of office employees in the ...,65.331,NBC,,2005-03-24,20,Ended,Scripted,8.1,572
4,http://www.nbc.com/parks-and-recreation/,8592,False,1,4,Parks and Recreation,125,7,Hilarious ensemble comedy that follows Leslie ...,18.586,NBC,Universal Television,2009-04-09,22,Ended,Scripted,7.9,425


In [ ]:
merged = pd.merge(shows2, df1)